<a href="https://colab.research.google.com/github/Clinda02/financial_data_analysis/blob/main/Finance_news_scarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요
* 네이버 증권의 `뉴스` 탭의 최신 10일치 기사를 수집하여 데이터 프레임으로 생성, 데이터프레임을 `article.csv` 로 저장
* 사이트 주소 : https://finance.naver.com/news/mainnews.naver?date=2024-10-25


-----

**네이버 증권 뉴스 탭**
* 날짜별로 페이지가 구분되어 있고, 날짜를 변경할 경우, URL의 date의 속성값 변경
  * `https://finance.naver.com/news/mainnews.naver?date=2024-10-25`

* 각 날짜에는 페이지네이션으로 페이지가 변경됨
  * `https://financenaver.com/news/mainnews.naver?date=2024-10-25&page=3`
* 페이지네이션은 고려하지 않고, 첫 페이지에 있는 데이터만 수집

**데이터 프레임 구조**
* `기사제목, 본문URL, 기사본문, 날짜` 4개 칼럼으로 구성

# 신문기사 HTML 불러오기

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# 뉴스 스크랩
url = "https://finance.naver.com/news/mainnews.naver?date=2024-10-31"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'lxml')

- 데이터 프레임 구조

  - **기사제목**:  `<dd class="articleSubject">`안에 텍스트
  - **본문 URL**: `<dd class="articleSubject">` 안에 `<a>` 태그
  - **기사 본문**: `<div id="newsct_article" class="newsct_article _article_body">`
  - **날짜**: URL에서 가져오기

In [2]:
# 기사제목 리스트 가져오기
titles = soup.find_all('dd', {'class':'articleSubject'})
titles[0]

<dd class="articleSubject">
<a href="/news/news_read.naver?article_id=0005493204&amp;office_id=277&amp;mode=mainnews&amp;type=&amp;date=2024-10-31&amp;page=1">코스피, 외인 '팔자'에 2550선까지 밀려…코스닥은 상승</a>
</dd>

In [3]:
titles[0].text.strip()

"코스피, 외인 '팔자'에 2550선까지 밀려…코스닥은 상승"

# 상세 뉴스 본문 읽어오기

## 상세 뉴스 URL 읽어오기

In [4]:
# 첫번째 기사의 상세페이지 URL
titles[0].find('a').get('href')

'/news/news_read.naver?article_id=0005493204&office_id=277&mode=mainnews&type=&date=2024-10-31&page=1'

In [5]:
news_url = 'https://finance.naver.com' + titles[0].find('a').get('href')
news_url

'https://finance.naver.com/news/news_read.naver?article_id=0005493204&office_id=277&mode=mainnews&type=&date=2024-10-31&page=1'

In [6]:
# 상세 뉴스 페이지
news_response = requests.get(news_url)
news_soup = BeautifulSoup(news_response.text, 'lxml')
news_soup

<html><head><script>top.location.href='https://n.news.naver.com/mnews/article/277/0005493204';</script>
</head></html>

redirect 주소를 반환하고 있으므로, redirecet 주소를 사용하여 다시 상세 뉴스 페이지에 접근

In [7]:
news_url2 = news_soup.find('script').text.split("'")[1]

## 상세 뉴스 페이지 읽어오기

In [8]:
# 상세 뉴스페이지 재호출
news_response2 = requests.get(news_url2)

news_soup = BeautifulSoup(news_response2.text, 'lxml')

## 상세 뉴스페이지 본문 파싱하기

In [9]:
# 뉴스 본문
# 기사 본문: <div id="newsct_article" class="newsct_article _article_body">
news_soup.find('div',{'class', 'newsct_article _article_body'}).text.strip()

'코스피가 외국인의 매도세에 하락하며 2550선까지 밀렸다. 반면 코스닥은 강보합 마감했다.\n\n\n\n[이미지출처=연합뉴스]코스피는 31일 전거래일 대비 37.64포인트(1.45%) 내린 2556.15에 마감했다.투자자별로는 개인과 기관이 각각 5642억원, 3300억원을 순매수했다. 반면 외국인은 8630억원을 순매도했다.업종별로는 섬유·의복이 2.80% 뛰었다. 또 음식료품, 의료정밀 등이 1% 이상 올랐다. 반면 유통업이 3.04% 내렸다. 이 외 철강및금속, 기계, 의약품, 금융업 등이 2% 이상 하락했다.시가총액 상위종목 중 삼성전자는 전거래일 대비 100원(0.17%) 오른 5만9200원에 거래를 마감했다. 반면 SK하이닉스는 4.46% 내렸다. 또 신한지주가 5.87% 급락했다. 이 외 삼성바이오로직스, 현대차, 기아, 셀트리온 등은 2% 이상 하락했다.이날 코스피에서는 미래산업이 상한가를 기록하며 473개 종목이 올랐으며 하한가 없이 405개 종목이 밀렸다. 62개 종목은 보합이었다.코스닥은 전거래일 대비 4.87포인트(0.66%) 오른 743.06에 거래를 마감했다.투자자별로는 외국인이 1493억원을 순매수했다. 반면 개인과 기관이 각각 1370억원과 41억원을 순매도했다.업종별로는 비금속, 화학, 운송, 제약 등이 1% 이상 올랐다. 반면 금융이 1.26% 하락했다. 이 외 종이·목재, 유통, 기타서비스, 의료·정밀기기 등이 약보합 마감했다.시가총액 상위종목에서 알테오젠은 전거래일 대비 2500원(0.66%) 내린 37만6500원에 거래를 마감했다. 에코프로는 1.74% 내렸다. 반면 삼천당제약이 6.20% 급등했다. HPSP는 4.77% 뛰었다. 이 외 리가켐바이오, 엔켐, 셀트리온제약 등은 강보합 마감했다.코스닥에서는 비투엔이 상한가를 기록하며 882개 종목이 상승했다. 반면 하한가 없이 678개 종목이 밀렸으며 122개 종목은 보합으로 마감했다.'

# 반복문 사용하여 함수로 생성하기

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
import time

In [11]:
 def get_news_items(html):

  # 기사 리스트 가져오기
  titles = html.find_all('dd', {'class' :'articleSubject'})

  title_list = []
  url_list =[]
  content_list = []

  for t in titles:
    # 뉴스 기사 제목
    title = t.text.strip()
    # print(f"제목 : {title}")

    # 뉴스 기사의 상세페이지 url
    news_url = 'https://finance.naver.com' + t.find('a').get('href')
    # print(f"url : {news_url}")

    # 상세페이지 request
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.text, 'lxml')

    # 리다이렉트 주소 파싱
    news_url2 = news_soup.find('script').text.split("'")[1]

    # 상세페이지 request
    news_response2 = requests.get(news_url2)
    news_soup = BeautifulSoup(news_response2.text, 'lxml')

    # 뉴스 본문
    content = news_soup.find('div', {'class':'newsct_article _article_body'}).text.strip()
    # print(f"본문 : {content}")

    # 리스트 값 저장
    title_list.append(title)
    url_list.append(news_url2)
    content_list.append(content)

  # 데이터프레임 저장
  df = pd.DataFrame({
        '기사제목' : title_list,
        '본문url' : url_list,
        '기사본문' : content_list
    })
  return df


In [12]:
news_df = pd.DataFrame()

for n in range(2):
  # 최근 n일치 데이터 지정
  date = datetime.date.today() - datetime.timedelta(days = n)
  url = f"https://finance.naver.com/news/mainnews.naver?date={date}"
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'lxml')
  time.sleep(1)

  # 해당일 신문기사 스크랩 데이터프레임 반환
  temp_df = get_news_items(soup)
  temp_df['date']= date
  news_df = pd.concat([news_df, temp_df], ignore_index =True).copy()

In [13]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기사제목    40 non-null     object
 1   본문url   40 non-null     object
 2   기사본문    40 non-null     object
 3   date    40 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [14]:
news_df.head()

,기사제목,본문url,기사본문,date
0,"코스피, 외인 '팔자'에 2550선까지 밀려…코스닥은 상승",https://n.news.naver.com/mnews/article/277/000...,코스피가 외국인의 매도세에 하락하며 2550선까지 밀렸다. 반면 코스닥은 강보합 마...,2024-10-31
1,"6만원대 터치했지만…삼성전자, 뒷심부족에 '찔끔' 상승",https://n.news.naver.com/mnews/article/018/000...,[특징주][이데일리 김인경 기자] 삼성전자(005930)가 31일 장 중 6만원을 ...,2024-10-31
2,"정말 '왕'이었나? 이정재… 래몽래인 '경영권 분쟁' 완승, 사명도 변경",https://n.news.naver.com/mnews/article/417/000...,이정재가 최대주주로 있는 아티스트유나이티드가 콘텐츠기업 래몽래인과의 경영권 갈등에서...,2024-10-31
3,"삼전 HBM 기대감에도…""6만전자 쉽지않네"" [장 안의 화제]",https://n.news.naver.com/mnews/article/215/000...,※ 한국경제TV 생방송 성공투자오후증시는 매일 오후 2시에 방영됩니다. 투자 판단은...,2024-10-31
4,외국인 매수세에 740선 사수…반도체 소부장↑[코스닥 마감],https://n.news.naver.com/mnews/article/018/000...,"외국인 ‘사자’ vs 개인·기관 ‘팔자’“실적, 이슈에 따른 종목 장세”업종별 혼조...",2024-10-31


In [15]:
news_df.tail()

,기사제목,본문url,기사본문,date
35,을지로 유안타증권 빌딩 5920억 PF 성공,https://n.news.naver.com/mnews/article/014/000...,"캡스톤운용, 2022년 인수 후 쾌거 \n\n\n\n유안타증권 본사 전경 유안타증권...",2024-10-30
36,"강남 르메르디앙 개발, 내년 하반기 '첫삽'…대출 9500억, 내년 6월 만기",https://n.news.naver.com/mnews/article/018/000...,웰스어드바이저스·현대건설·마스턴운용·메리츠 '컨소''창의·혁신디자인' 시범사업지…서...,2024-10-30
37,"영풍·MBK ""고려아연 유증 저지 위한 모든 법적수단 강구""",https://n.news.naver.com/mnews/article/015/000...,김광일 MBK 파트너스 부회장(가운데). 사진=뉴스1최윤범 고려아연 회장 측이 3...,2024-10-30
38,"""미장은 장기투자용, 국장은 단타용""…MZ들은 국장을 왜 떠나나",https://n.news.naver.com/mnews/article/421/000...,"[NIF2024]MZ토론회…국장은 불확실·불안정, 미장은 성장성 주목 국장 대한 애...",2024-10-30
39,진흙탕 싸움된 고려아연 사태 … 금감원 칼 뺐다,https://n.news.naver.com/mnews/article/009/000...,"금감원, 고려아연 유증 제동시세조종 정황도 포착한 당국불공정행위 여부 고강도 조사주...",2024-10-30


#  CSV 파일로 저장

In [16]:
news_df.to_csv('article.csv')